In [1]:
import sys
sys.path.append("..")
import pickle

In [2]:
from games.abstract_poker import AbstractPokerRootChanceGameState
from games.algorithms import ChanceSamplingCFR, VanillaCFR
from common.constants import card_dealing, results_map
import numpy as np
import pandas as pd

In [3]:
# Hyperparameters
# _MAX_HAND_STRENGTH = 3
_MAX_TURNS = 2
_NUM_TRAIN_ITERS = 50_000

In [67]:
# card_dealing(_MAX_HAND_STRENGTH)

# Train

In [68]:
bots = []

for i in range(2,11):
    root = AbstractPokerRootChanceGameState(max_hand_strength = i, 
                                        max_turns = _MAX_TURNS,
                                        report = False)
    chance_sampling_cfr = ChanceSamplingCFR(root)
    chance_sampling_cfr.run(iterations = _NUM_TRAIN_ITERS)
    chance_sampling_cfr.compute_nash_equilibrium()

    # vanilla cfr
#     vanilla_cfr_g3 = VanillaCFR(root)
#     vanilla_cfr_g3.run(iterations = _NUM_TRAIN_ITERS)
#     vanilla_cfr_g3.compute_nash_equilibrium()

    bots.append(chance_sampling_cfr)

In [80]:
for b in bots:
    print(f"Value of game for granularity {b.root.max_hand_strength}: {b.value_of_the_game():.5f}")

Value of game for granularity 2: -0.00022
Value of game for granularity 3: -0.02202
Value of game for granularity 4: -0.04648
Value of game for granularity 5: -0.03589
Value of game for granularity 6: -0.04565
Value of game for granularity 7: -0.05528
Value of game for granularity 8: -0.04921
Value of game for granularity 9: -0.05261
Value of game for granularity 10: -0.05370


# Competition

In [71]:
from games.competition_abstract_poker import CompetitionAbstractPoker
from copy import deepcopy

In [73]:
bots[0].root.max_hand_strength

2

In [81]:
results = []
rounds_per_competition = 100_000

for b1 in bots:
    for b2 in bots:
        g1 = b1.root.max_hand_strength
        g2 = b2.root.max_hand_strength
        c = CompetitionAbstractPoker(deepcopy(b1), deepcopy(b2), report = False)
        result = c.compete(rounds = rounds_per_competition)
        results.append([g1,g2,result])
        print(f"{g1} vs {g2}. Result is {result}")

2 vs 2. Result is -632
2 vs 3. Result is -5369
2 vs 4. Result is -6321
2 vs 5. Result is -7475
2 vs 6. Result is -4643
2 vs 7. Result is -5490
2 vs 8. Result is -7554
2 vs 9. Result is -8497
2 vs 10. Result is -8463
3 vs 2. Result is 1684
3 vs 3. Result is -4996
3 vs 4. Result is -7330
3 vs 5. Result is -5720
3 vs 6. Result is -6747
3 vs 7. Result is -7406
3 vs 8. Result is -6316
3 vs 9. Result is -7909
3 vs 10. Result is -5913
4 vs 2. Result is 2433
4 vs 3. Result is -5398
4 vs 4. Result is -7297
4 vs 5. Result is -5995
4 vs 6. Result is -7861
4 vs 7. Result is -7731
4 vs 8. Result is -7825
4 vs 9. Result is -8988
4 vs 10. Result is -7735
5 vs 2. Result is 3752
5 vs 3. Result is -3814
5 vs 4. Result is -4396
5 vs 5. Result is -4237
5 vs 6. Result is -4756
5 vs 7. Result is -4844
5 vs 8. Result is -5698
5 vs 9. Result is -5231
5 vs 10. Result is -3933
6 vs 2. Result is 403
6 vs 3. Result is -2898
6 vs 4. Result is -7081
6 vs 5. Result is -5471
6 vs 6. Result is -6009
6 vs 7. Result is 

In [83]:
results_df = pd.DataFrame(results, columns = ['player1', 'player2', 'result'])\
    .assign(total_rounds = rounds_per_competition)

In [86]:
results_df

,player1,player2,result,total_rounds
0,2,2,-632,100000
1,2,3,-5369,100000
2,2,4,-6321,100000
3,2,5,-7475,100000
4,2,6,-4643,100000
...,...,...,...,...
76,10,6,-6879,100000
77,10,7,-7084,100000
78,10,8,-5391,100000
79,10,9,-6749,100000


In [84]:
results_df.to_csv("bot_competition_by_granularity.csv")